In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../../data/cleansed/bangkok_traffy_clean.csv')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 778255 entries, 0 to 778254
Data columns (total 17 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   ticket_id         778254 non-null  object 
 1   type              778158 non-null  object 
 2   organization      777689 non-null  object 
 3   comment           778254 non-null  object 
 4   photo             778140 non-null  object 
 5   photo_after       634024 non-null  object 
 6   latitude          778255 non-null  float64
 7   longitude         778255 non-null  float64
 8   address           778253 non-null  object 
 9   province          778061 non-null  object 
 10  district          777695 non-null  object 
 11  subdistrict       777690 non-null  object 
 12  state             778255 non-null  object 
 13  star              272358 non-null  float64
 14  count_reopen      778255 non-null  int64  
 15  timestamp_dt      778255 non-null  object 
 16  last_activity_dt  77

,ticket_id,type,organization,comment,photo,photo_after,latitude,longitude,address,province,district,subdistrict,state,star,count_reopen,timestamp_dt,last_activity_dt
0,2021-FYJTFP,{ความสะอาด},เขตบางซื่อ,ขยะเยอะ,https://storage.googleapis.com/traffy_public_b...,NaN,13.81865,100.53084,12/14 ถนน กรุงเทพ- นนทบุรี แขวง บางซื่อ เขตบาง...,กรุงเทพมหานคร,NaN,NaN,เสร็จสิ้น,NaN,0,2021-09-03T12:51:09.000Z,2022-06-04T15:34:14.000Z
1,2021-CGPMUN,"{น้ำท่วม,ร้องเรียน}","เขตประเวศ,ฝ่ายโยธา เขตประเวศ",น้ำท่วมเวลาฝนตกและทะลุเข้าบ้านเดือดร้อนมากทุกๆ...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,13.67891,100.66709,189 เฉลิมพระเกียรติ ร.9 แขวง หนองบอน เขต ประเว...,กรุงเทพมหานคร,ประเวศ,หนองบอน,เสร็จสิ้น,4.0,0,2021-09-19T14:56:08.000Z,2022-06-21T08:21:09.000Z
2,2021-7XATFA,{สะพาน},เขตสาทร,สะพานลอยปรับปรุงไม่เสร็จตามกำหนด ปากซอย สาทร12,https://storage.googleapis.com/traffy_public_b...,NaN,13.72060,100.52649,191/1 ถนน สาทรเหนือ แขวง สีลม เขตบางรัก กรุงเท...,กรุงเทพมหานคร,สาทร,ยานนาวา,เสร็จสิ้น,NaN,0,2021-09-26T05:03:52.000Z,2022-06-06T01:17:12.000Z
3,2021-9U2NJT,{น้ำท่วม},"เขตบางซื่อ,ฝ่ายโยธา เขตบางซื่อ",น้ำท่วม,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,13.81853,100.53099,12/14 ถนน กรุงเทพ- นนทบุรี แขวง บางซื่อ เขตบาง...,กรุงเทพมหานคร,NaN,NaN,เสร็จสิ้น,NaN,0,2021-10-14T10:45:27.000Z,2022-09-08T08:35:43.000Z
4,2021-DVEWYM,"{น้ำท่วม,ถนน}","เขตลาดพร้าว,ฝ่ายโยธา เขตลาดพร้าว",ซอยลาดพร้าววังหิน 75 ถนนลาดพร้าววังหิน แขวงลาด...,https://storage.googleapis.com/traffy_public_b...,NaN,13.82280,100.59165,702 ถ. ลาดพร้าววังหิน แขวงลาดพร้าว เขตลาดพร้าว...,กรุงเทพมหานคร,ลาดพร้าว,ลาดพร้าว,เสร็จสิ้น,5.0,0,2021-12-09T12:29:08.000Z,2022-08-12T07:18:44.000Z


# Data Preprocessing

In [3]:
from sklearn.preprocessing import MultiLabelBinarizer 

df_subset = df[['comment', 'type']].dropna().copy()

label_lists = (
    df_subset['type']
    .astype(str)
    .str.strip('{}')
    .str.split(',')
    .apply(lambda items: [item.strip() for item in items if item.strip()])
)
valid_mask = label_lists.map(bool)
comments = df_subset.loc[valid_mask, 'comment']
labels = label_lists.loc[valid_mask].tolist()

mlb = MultiLabelBinarizer()
Y = mlb.fit_transform(labels)

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(
    comments, Y, test_size=0.2, random_state=2025
)

X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

((532511,), (532511, 24), (133128,), (133128, 24))

# Classification ML

In [13]:
from pythainlp.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report, f1_score

def thai_tokenizer(text):
    # engine='newmm' เป็น dictionary-based ที่นิยมที่สุด
    return word_tokenize(text, engine='newmm')

def get_classification_report(Y_test, Y_pred, mlb):
    print("Classification Report:")
    print(classification_report(Y_test, Y_pred, target_names=mlb.classes_))

def get_f1_score(Y_test, Y_pred):
    macro_f1 = f1_score(Y_test, Y_pred, average='macro')
    micro_f1 = f1_score(Y_test, Y_pred, average='micro')
    weighted_f1 = f1_score(Y_test, Y_pred, average='weighted')

    print(f"Macro F1 score: {macro_f1:.4f}")
    print(f"Micro F1 score: {micro_f1:.4f}")
    print(f"Weighted F1 score: {weighted_f1:.4f}")

### 1. Logistic Regression (Baseline Model)
โมเดลพื้นฐานที่เน้นความเรียบง่าย รวดเร็ว และใช้ทรัพยากรน้อย 
ใช้เป็นเกณฑ์มาตรฐาน (Baseline) เพื่อเปรียบเทียบกับโมเดลที่ซับซ้อนกว่า

🛠️ **Techniques & Preprocessing**
* **Tokenization:** ใช้ `PyThaiNLP` (engine=`newmm`) ในการตัดคำภาษาไทย เนื่องจากภาษาไทยไม่มีเว้นวรรค
* **Vectorization:** ใช้ **TF-IDF** เพื่อแปลงข้อความให้เป็นตัวเลข โดยให้ค่าน้ำหนักกับคำที่มีความสำคัญในเอกสารนั้นๆ
* **Multi-label Strategy:** ใช้เทคนิค **One-vs-Rest** ซึ่งจะสร้าง Binary Classifier แยกกันสำหรับแต่ละ Tag แล้วนำผลมารวมกัน

⚙️ **Hyperparameters & Configuration**
* **TF-IDF Settings:**
    * `ngram_range=(1, 2)`: เก็บทั้งคำเดี่ยว (Unigram) และคำคู่ (Bigram) เพื่อจับบริบทสั้นๆ เช่น "ไม่" + "สะอาด"
    * `min_df=5`: ตัดคำที่ปรากฏน้อยกว่า 5 ครั้งทิ้ง (ลด Noise)
    * `max_features=5000`: จำกัดจำนวนคำศัพท์สูงสุด 5,000 คำ เพื่อไม่ให้ Model ใหญ่เกินไป
* **Model Settings:**
    * `solver='liblinear'`: เหมาะสำหรับข้อมูลขนาดเล็ก-กลาง และรองรับ L1/L2 Regularization
    * `C=1.0`: ค่า Regularization strength (ค่ามาตรฐาน)

In [ ]:
# ---------------------------------------------------------
# Classification ML - LogisticRegression
# ---------------------------------------------------------

lr_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(
        tokenizer=thai_tokenizer, 
        ngram_range=(1, 2),    
        min_df=5,              
        max_features=5000     
    )),
    ('clf', OneVsRestClassifier(LogisticRegression(solver='liblinear', C=1.0)))
])

lr_pipeline.fit(X_train, Y_train)

c:\Users\USER\miniconda3\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


### 2. XGBoost (Extreme Gradient Boosting)
โมเดลประเภท Tree-based Ensemble สามารถจับ Pattern ที่ซับซ้อนกว่า Linear Model

**🛠️ Techniques & Preprocessing**
* **Input:** รับค่าจาก TF-IDF Vectorizer (เหมือน Logistic Regression)
* **Hardware Acceleration:** ใช้ **GPU** ในการประมวลผล เพื่อความรวดเร็วในการสร้าง Decision Tree หลายร้อยต้น

**⚙️ Hyperparameters & Configuration**
* **Tree Method:** `hist` (Histogram-based) เพื่อประสิทธิภาพสูงสุดบน GPU
* **Device:** `cuda` (รันบน GPU)
* **Objective:** `binary:logistic` (เนื่องจากใช้ OneVsRest แต่ละโมเดลย่อยจึงมองเป็น Binary)
* **Tuning:**
    * `n_estimators=100`: จำนวนต้นไม้
    * `learning_rate=0.1`: อัตราการเรียนรู้
    * `max_depth=6`: ความลึกสูงสุดของต้นไม้ (กัน Overfitting)

In [20]:
# ---------------------------------------------------------
# Classification ML - XGBoost
# ---------------------------------------------------------

from xgboost import XGBClassifier

# สร้าง Pipeline เหมือนเดิม แต่เปลี่ยนตัวจบเป็น XGBoost
xgb_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(
        tokenizer=thai_tokenizer, 
        ngram_range=(1, 2),
        min_df=5,
        max_features=5000
    )),
    ('clf', OneVsRestClassifier(
        XGBClassifier(
            tree_method='hist',      # ใช้ GPU (เวอร์ชันใหม่ใช้ 'hist' + device)
            device='cuda',           # 🚀 สั่งให้รันบน RTX 3060
            n_estimators=100,
            learning_rate=0.1,
            max_depth=6,
            objective='binary:logistic' # Multi-label คือ Binary หลายตัวรวมกัน
        )
    ))
])

print("🚀 Training XGBoost...")
xgb_pipeline.fit(X_train, Y_train)
print("✅ Done!")

🚀 Training XGBoost...
✅ Done!


### 3. Linear SVC (Support Vector Classifier) ⚔️
โมเดลกลุ่ม Support Vector Machine (SVM) ที่ปรับจูนมาเพื่อการทำงานแบบ Linear โดยเฉพาะ สามารถจัดการกับข้อมูลที่มีมิติสูง (High Dimensional Data) ได้ดี และมักให้ความแม่นยำสูงกว่า Logistic Regression

**🛠️ Techniques & Preprocessing**
* **Vectorization:** ใช้ TF-IDF เพื่อแปลงข้อความ เป็น Vector ขนาด 5,000 มิติ
* **Margin Maximization:** หลักการของ SVC คือการหา "เส้นแบ่ง" (Hyperplane) ที่มีระยะห่าง (Margin) ระหว่างข้อมูลแต่ละกลุ่มให้มากที่สุด ทำให้โมเดลมีความทนทานต่อ Noise และไม่ Overfit ง่าย
* **Multi-label Strategy:** ใช้ **One-vs-Rest** เหมือนกับโมเดลอื่นๆ เพื่อจัดการกับโจทย์ที่มีหลายคำตอบ โดยสร้างเส้นแบ่งแยกสำหรับแต่ละ Tag

**⚙️ Hyperparameters & Configuration**
* **Penalty:** `l2` (Ridge regularization) เป็นค่ามาตรฐานที่ช่วยควบคุมไม่ให้โมเดลซับซ้อนเกินไป
* **Loss:** `squared_hinge` เป็น Loss Function ที่เหมาะสำหรับการหา Margin สูงสุด
* **Dual:** `auto` (ให้ Library เลือกเอง) ขึ้นอยู่กับจำนวนข้อมูลและ Feature เพื่อความเร็วสูงสุด
* **C:** `1.0` ค่า Regularization Parameter

In [23]:
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import Pipeline

# ใช้ Pipeline เดิม แต่เปลี่ยนตัวจบเป็น LinearSVC
svc_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(tokenizer=thai_tokenizer, ngram_range=(1, 2), max_features=5000)),
    ('clf', OneVsRestClassifier(LinearSVC(dual='auto', random_state=42)))
])

print("🚀 Training Linear SVC...")
svc_pipeline.fit(X_train, Y_train)
print("✅ Done!")

🚀 Training Linear SVC...
✅ Done!


# Model Evaluation 

In [ ]:
# ---------------------------------------------------------
# Prediction 
# ---------------------------------------------------------

lr_pred = lr_pipeline.predict(X_test)
xgb_pred = xgb_pipeline.predict(X_test)
svc_pred = svc_pipeline.predict(X_test)

In [ ]:
# ---------------------------------------------------------
# Evaluation Metrics
# ---------------------------------------------------------

print("Logistic Regression Performance:")
get_f1_score(Y_test, lr_pred)

print("\nXGBoost Performance:")
get_f1_score(Y_test, xgb_pred)

print("\nLinear SVC Performance:")
get_f1_score(Y_test, svc_pred)

Macro F1 score: 0.8897
Micro F1 score: 0.9256
Weighted F1 score: 0.9250
